## Importing the libraries

In [1]:
import os
import pandas as pd
import numpy as np
import json
import spotipy
import spotipy.oauth2 as oauth2
from spotipy.oauth2 import SpotifyOAuth,SpotifyClientCredentials
import yaml
import re
from tqdm import tqdm
import multiprocessing as mp
import time
import random
import datetime
import glob

In [34]:
auth_manager = SpotifyClientCredentials(client_id=cl_id,
                                        client_secret=secret)
sp = spotipy.client.Spotify(auth_manager=auth_manager)

## Importing the dataset

In [ ]:
def loop_slices(path, num_slices=20):
  cnt = 0
  cnt1 = 0
  mpd_playlists = []
  unique_tracks= pd.DataFrame()
  filenames = os.listdir(path)
  for fname in tqdm(sorted(filenames, key=len)):
    if fname.startswith("mpd.slice.") and fname.endswith(".json"):
      cnt += 1
      fullpath = os.sep.join((path, fname))
      f = open(fullpath)
      js = f.read()
      f.close()
      current_slice = json.loads(js)
      for playlist in current_slice['playlists']:
        cnt1 +=1
        mpd_playlists.append(playlist)
        if cnt1 == 1000:
          cnt1=0
          temp=pd.DataFrame(mpd_playlists)
          temp=temp.explode('tracks')
          temp=pd.DataFrame(temp['tracks'].apply(pd.Series))
          unique_tracks=pd.concat([unique_tracks,temp],axis=0)
          unique_tracks.drop_duplicates(subset=['track_uri'],inplace=True)
          mpd_playlists = []
      if cnt == num_slices:
        break
  return unique_tracks
path = 'spotify_million_playlist_dataset/data/'

In [ ]:
df = loop_slices(path, num_slices=1000)
df.to_csv('data/1m.csv')

100%|█████████▉| 1000/1001 [1:15:03<00:04,  4.50s/it]


OSError: Cannot save file into a non-existent directory: 'data'

In [ ]:
df

,Unnamed: 0,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook
1,0,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone
2,0,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit)
3,0,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified
4,0,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot
...,...,...,...,...,...,...,...,...,...
2262287,981,19,Arab Strap,spotify:track:2wBZrBqWQ4eIhShITfUJ4c,spotify:artist:6g8Jqb5JMfv92eB2r0awTN,My Favourite Muse,spotify:album:7maGHx2VJGTBPczcWT9JVm,303106,Philophobia
2262288,985,2,Various Artists,spotify:track:2t7fotSpsiWHpSTySbSNZg,spotify:artist:0LyfQWJT6nXafLPZqxe9Of,The Stars and Stripes Forever,spotify:album:5HuTwuPKjQKNpmLmmi4Njx,314644,My Kids Salute Our Troops
2262289,998,3,Faragó,spotify:track:0EW4RltERtn276lOi0DXQj,spotify:artist:6x3HJm6n40OUW2ZcmttBxQ,Killer,spotify:album:5aauU2AK4iG01BxIchws8Q,297176,Downtown / Archipelago
2262290,998,71,Crazy Fool,spotify:track:5RiCId3jJs8D9g6Fv4A1Su,spotify:artist:4UrrAgW6WW7UfZ760eyHe7,Robin Hood,spotify:album:2MUG8V88S5mazH8n45AKuZ,278973,Train of Thought


In [ ]:
df.to_csv('1m.csv')

## Splitting

In [82]:
df=pd.read_csv('1m.csv')

In [83]:
df.columns

Index(['Unnamed: 0', 'album', 'pos', 'artist_name', 'track_uri', 'artist_uri',
       'track_name', 'album_uri', 'duration_ms', 'album_name'],
      dtype='object')

In [84]:
df.drop(columns=['Unnamed: 0'], inplace=True)

In [85]:
df

,album,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook
1,0,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone
2,0,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit)
3,0,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified
4,0,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot
...,...,...,...,...,...,...,...,...,...
2262287,981,19,Arab Strap,spotify:track:2wBZrBqWQ4eIhShITfUJ4c,spotify:artist:6g8Jqb5JMfv92eB2r0awTN,My Favourite Muse,spotify:album:7maGHx2VJGTBPczcWT9JVm,303106,Philophobia
2262288,985,2,Various Artists,spotify:track:2t7fotSpsiWHpSTySbSNZg,spotify:artist:0LyfQWJT6nXafLPZqxe9Of,The Stars and Stripes Forever,spotify:album:5HuTwuPKjQKNpmLmmi4Njx,314644,My Kids Salute Our Troops
2262289,998,3,Faragó,spotify:track:0EW4RltERtn276lOi0DXQj,spotify:artist:6x3HJm6n40OUW2ZcmttBxQ,Killer,spotify:album:5aauU2AK4iG01BxIchws8Q,297176,Downtown / Archipelago
2262290,998,71,Crazy Fool,spotify:track:5RiCId3jJs8D9g6Fv4A1Su,spotify:artist:4UrrAgW6WW7UfZ760eyHe7,Robin Hood,spotify:album:2MUG8V88S5mazH8n45AKuZ,278973,Train of Thought


In [51]:
albums = df["artist_name"].unique()
len(albums)

287740

In [ ]:
unique_tracks_df = df.drop_duplicates(subset=['track_name'])
unique_tracks_df

,album,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook
1,0,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone
2,0,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit)
3,0,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified
4,0,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot
...,...,...,...,...,...,...,...,...,...
2262282,948,10,SNAP!,spotify:track:1qjEdWY2RHoVbiCYsbwi26,spotify:artist:2FrKQPjJe4pVMZOgm0ESOx,Exterminate - Michael Gray Remix,spotify:album:7GthPhu4rupbUY0vOQKxJW,428571,SNAP! The Power Greatest Hits
2262283,959,2,Suicidal Tendencies,spotify:track:2kKfbRXlBmaX4A8Lii0SE6,spotify:artist:3WPKDlucMsXH6FC1XaclZC,Born To Be Cyco,spotify:album:5Q9XRmD1Z9IVF4Tlz0MsSK,144013,No Mercy Fool! / The Suicidal Family
2262284,971,35,Disturbing Tha Peace,spotify:track:2rFbG6ltIDAIdawGMjW1oj,spotify:artist:3FsQX7FsrZAzTU9ah45P7O,Break Sumthin',spotify:album:1HjiZwq8HvkzLchkB2XcyV,277560,Golden Grain
2262287,981,19,Arab Strap,spotify:track:2wBZrBqWQ4eIhShITfUJ4c,spotify:artist:6g8Jqb5JMfv92eB2r0awTN,My Favourite Muse,spotify:album:7maGHx2VJGTBPczcWT9JVm,303106,Philophobia


In [ ]:
unique_tracks_df.to_csv('unique_songs_dataset.csv' , index=False)

In [ ]:
unique_tracks_df = pd.read_csv('unique_songs_dataset.csv')

In [ ]:
unique_tracks_df

,album,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook
1,0,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone
2,0,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit)
3,0,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified
4,0,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot
...,...,...,...,...,...,...,...,...,...
1483754,948,10,SNAP!,spotify:track:1qjEdWY2RHoVbiCYsbwi26,spotify:artist:2FrKQPjJe4pVMZOgm0ESOx,Exterminate - Michael Gray Remix,spotify:album:7GthPhu4rupbUY0vOQKxJW,428571,SNAP! The Power Greatest Hits
1483755,959,2,Suicidal Tendencies,spotify:track:2kKfbRXlBmaX4A8Lii0SE6,spotify:artist:3WPKDlucMsXH6FC1XaclZC,Born To Be Cyco,spotify:album:5Q9XRmD1Z9IVF4Tlz0MsSK,144013,No Mercy Fool! / The Suicidal Family
1483756,971,35,Disturbing Tha Peace,spotify:track:2rFbG6ltIDAIdawGMjW1oj,spotify:artist:3FsQX7FsrZAzTU9ah45P7O,Break Sumthin',spotify:album:1HjiZwq8HvkzLchkB2XcyV,277560,Golden Grain
1483757,981,19,Arab Strap,spotify:track:2wBZrBqWQ4eIhShITfUJ4c,spotify:artist:6g8Jqb5JMfv92eB2r0awTN,My Favourite Muse,spotify:album:7maGHx2VJGTBPczcWT9JVm,303106,Philophobia


In [ ]:
df = unique_tracks_df

chunk_size = 100000

for i, start in enumerate(range(0, len(df), chunk_size), start=1):
    chunk_df = df.iloc[start:start + chunk_size]
    output_file_path = f'unique_songs_dataset_{i}.csv'

    chunk_df.to_csv(output_file_path, index=False)
    print(f"Saved {output_file_path} with {len(chunk_df)} rows.")

Saved unique_songs_dataset_1.csv with 100000 rows.
Saved unique_songs_dataset_2.csv with 100000 rows.
Saved unique_songs_dataset_3.csv with 100000 rows.
Saved unique_songs_dataset_4.csv with 100000 rows.
Saved unique_songs_dataset_5.csv with 100000 rows.
Saved unique_songs_dataset_6.csv with 100000 rows.
Saved unique_songs_dataset_7.csv with 100000 rows.
Saved unique_songs_dataset_8.csv with 100000 rows.
Saved unique_songs_dataset_9.csv with 100000 rows.
Saved unique_songs_dataset_10.csv with 100000 rows.
Saved unique_songs_dataset_11.csv with 100000 rows.
Saved unique_songs_dataset_12.csv with 100000 rows.
Saved unique_songs_dataset_13.csv with 100000 rows.
Saved unique_songs_dataset_14.csv with 100000 rows.
Saved unique_songs_dataset_15.csv with 83759 rows.


In [ ]:
t_uri=df["track_uri"].unique()
a_uri=df["artist_uri"].unique()
name = df['track_name'].unique()

In [ ]:
len(name)

1483759

## Feature extraction

In [41]:
df = pd.read_csv('unique_songs_dataset_4.csv')

In [42]:
df

,album,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,766,14,Gera MXM,spotify:track:0eAKZIObmYM91semxYGXfB,spotify:artist:2hejA1Dkf8v8R0koF44FvW,Estoy Dando un Concierto,spotify:album:7kiDPvZND4Or7Ne2vSCAp8,185000,Estoy Dando un Concierto
1,766,15,Gera MXM,spotify:track:4wtWdEd3iRvI9Rcu2fF1ln,spotify:artist:2hejA1Dkf8v8R0koF44FvW,La Calle Que Me Gusta,spotify:album:71soyaxrVUCKn1S235wkpf,196000,La Calle Que Me Gusta
2,766,17,Los Choclok,spotify:track:1yyowcWPeWoDG00M7fivDa,spotify:artist:2iEDalkx4bi0VkRuW0QkYD,Aguardiente,spotify:album:6OI3xUm2zNMlhsngGiyTOX,297000,Sonido Místico
3,766,18,Los Choclok,spotify:track:3A4nkDh34S3QZdO56ac8Di,spotify:artist:2iEDalkx4bi0VkRuW0QkYD,Bachas,spotify:album:6OI3xUm2zNMlhsngGiyTOX,386000,Sonido Místico
4,767,10,Pries,spotify:track:2UV129WXyNO5iCWwYrQoOP,spotify:artist:1niLozUusxkiOr68fCKNny,Dreams Do Come True (feat. Natalie Tatum),spotify:album:2MDfGRUKIQoEcHG1empTvJ,232568,Cold Rose
...,...,...,...,...,...,...,...,...,...
99995,955,23,Morat,spotify:track:0XrcRy7nhZY4FnZQFuJ3Js,spotify:artist:5C4PDR4LnhZTbVnKWXuDKD,Ladrona,spotify:album:7tPz6T5lFv998HZc0O0Wh2,256146,Sobre El Amor Y Sus Efectos Secundarios
99996,955,24,Bebe,spotify:track:6Yf2PUq5aZxrCrkNMMIkHo,spotify:artist:2byU53ebtTxcAnIRE814Ts,Respirar,spotify:album:3GLyKimxM58Lu1N6FAEAEa,226242,Cambio de piel
99997,955,25,Bebe,spotify:track:4QB1P2dJVPQMgLrlxGtp47,spotify:artist:2byU53ebtTxcAnIRE814Ts,La Cuenta,spotify:album:3GLyKimxM58Lu1N6FAEAEa,289000,Cambio de piel
99998,955,30,Alejandro Fernandez,spotify:track:5t3LP7Qeog4pCfo2AiUrBj,spotify:artist:6sq1yF0OZEWA4xoXVKW1L9,Hoy Decidí Olvidarte (Aunque No Estés Conmigo)...,spotify:album:3DdhE6I0wvt9YsEbCW3K0q,255160,Confidencias Reales


In [43]:
t_uri=df["track_uri"].unique()
a_uri=df["artist_uri"].unique()
name = df['track_name'].unique()

In [44]:
len(t_uri)

100000

In [45]:
output_file = 'audio_features_4.csv'
e = 0
headers_written = False

with open(output_file, 'a') as f:
    for i in tqdm(range(0, len(t_uri), 100)):
        try:
            track_features = sp.audio_features(t_uri[i:i+100])
            track_df = pd.DataFrame(track_features)

            if not headers_written:
                track_df.to_csv(f, header=True, index=False)
                headers_written = True
            else:
                track_df.to_csv(f, header=False, index=False)

        except Exception as error:
            e += 1
            with open("audio_features_log.txt", "a") as r:
                r.write(f"{datetime.datetime.now().strftime('%d.%b %Y %H:%M:%S')}: {error}\n")
            time.sleep(3)
            continue

with open("audio_features_log.txt", "a") as r:
    r.write(f"{datetime.datetime.now().strftime('%d.%b %Y %H:%M:%S')} Total Number Of Errors: {e}\n")


100%|██████████| 1000/1000 [05:13<00:00,  3.19it/s]


In [46]:
output_file = 'track_features_4.csv'
f = open(output_file, 'a')
e = 0
headers_written = False

for i in tqdm(range(0, len(t_uri), 50)):
    try:
        track_features = sp.tracks(t_uri[i:i+50])

        for x in range(len(track_features['tracks'])):
            track_pop = pd.DataFrame([t_uri[i + x]], columns=['track_uri'])
            track_pop['release_date'] = track_features['tracks'][x]['album']['release_date']
            track_pop['popularity'] = track_features['tracks'][x]["popularity"]

            if not headers_written:
                track_pop.to_csv(f, header=True, index=False)
                headers_written = True
            else:
                track_pop.to_csv(f, header=False, index=False)

    except Exception as error:
        e += 1
        with open("track_features_log.txt", "a") as r:
            r.write(f"{datetime.datetime.now().strftime('%d.%b %Y %H:%M:%S')}: {error}\n")
        time.sleep(3)
        continue

with open("track_features_log.txt", "a") as r:
    r.write(f"{datetime.datetime.now().strftime('%d.%b %Y %H:%M:%S')} Total Number Of Errors: {e}\n")

f.close()

100%|██████████| 2000/2000 [09:03<00:00,  3.68it/s]


In [ ]:
f = open('data/artist_features.csv','a')
e=0
for i in tqdm(range(0,len(a_uri),50)):
    try:
        artist_features = sp.artists(a_uri[i:i+50])
        for x in range(50):
            artist_df=pd.DataFrame([a_uri[i+x]])
            artist_pop = artist_features['artists'][x]["popularity"]
            artist_genres = artist_features['artists'][x]["genres"]
            artist_df["artist_pop"] = artist_pop
            if artist_genres:
                artist_df["genres"] = " ".join([re.sub(' ','_',i) for i in artist_genres])
            else:
              artist_df["genres"] = "unknown"
            csv_data = artist_df.to_csv(header=False,index=False)
            f.write(csv_data)
    except Exception as error:
        e+=1
        r = open("artist_features.txt", "a")
        r.write(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S")+": "+str(error)+'\n')
        r.close()
        time.sleep(3)
        continue
r = open("artist_features.txt", "a")
r.write(datetime.datetime.now().strftime("%d.%b %Y %H:%M:%S")+" _________________________ "+"Total Number Of Errors : "+str(e)+" _________________________ "+'\n')
r.close()
f.close()

## Merging together

In [2]:
df = pd.read_csv('unique_songs_dataset.csv')

In [3]:
df

,album,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook
1,0,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone
2,0,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit)
3,0,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified
4,0,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot
...,...,...,...,...,...,...,...,...,...
1483754,948,10,SNAP!,spotify:track:1qjEdWY2RHoVbiCYsbwi26,spotify:artist:2FrKQPjJe4pVMZOgm0ESOx,Exterminate - Michael Gray Remix,spotify:album:7GthPhu4rupbUY0vOQKxJW,428571,SNAP! The Power Greatest Hits
1483755,959,2,Suicidal Tendencies,spotify:track:2kKfbRXlBmaX4A8Lii0SE6,spotify:artist:3WPKDlucMsXH6FC1XaclZC,Born To Be Cyco,spotify:album:5Q9XRmD1Z9IVF4Tlz0MsSK,144013,No Mercy Fool! / The Suicidal Family
1483756,971,35,Disturbing Tha Peace,spotify:track:2rFbG6ltIDAIdawGMjW1oj,spotify:artist:3FsQX7FsrZAzTU9ah45P7O,Break Sumthin',spotify:album:1HjiZwq8HvkzLchkB2XcyV,277560,Golden Grain
1483757,981,19,Arab Strap,spotify:track:2wBZrBqWQ4eIhShITfUJ4c,spotify:artist:6g8Jqb5JMfv92eB2r0awTN,My Favourite Muse,spotify:album:7maGHx2VJGTBPczcWT9JVm,303106,Philophobia


In [4]:
def merge_files_from_folder(folder_path):
    files = glob.glob(os.path.join(folder_path, '*.csv'))
    merged_list = []
    
    for file in files:
        df = pd.read_csv(file)
        merged_list.append(df)
    merged_df = pd.concat(merged_list, ignore_index=True)
    return merged_df

In [5]:
audio_features_folder = 'audio_features'
track_features_folder = 'track_features'

In [6]:
audio_features_df = merge_files_from_folder(audio_features_folder)
audio_features_df

/var/folders/rm/2kb48n595msf92h4d8417ly40000gn/T/ipykernel_14042/791740016.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)
/var/folders/rm/2kb48n595msf92h4d8417ly40000gn/T/ipykernel_14042/791740016.py:8: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.51,0.620,4.0,-6.269,1.0,0.0298,0.17200,0.000,0.0983,0.3160,136.203,audio_features,3KRIjXjUYoiweWF8x8Eg6v,spotify:track:3KRIjXjUYoiweWF8x8Eg6v,https://api.spotify.com/v1/tracks/3KRIjXjUYoiw...,https://api.spotify.com/v1/audio-analysis/3KRI...,256960.0,4.0
1,0.468,0.739,2.0,-6.123,1.0,0.0328,0.06120,0.000,0.2020,0.3710,155.984,audio_features,4TMy1kRMymO1iBNdqnMDGE,spotify:track:4TMy1kRMymO1iBNdqnMDGE,https://api.spotify.com/v1/tracks/4TMy1kRMymO1...,https://api.spotify.com/v1/audio-analysis/4TMy...,236387.0,3.0
2,0.38,0.440,4.0,-7.743,0.0,0.0612,0.44400,0.000,0.1510,0.1750,71.335,audio_features,5YltN2bsWbWMF0ka24K3Sn,spotify:track:5YltN2bsWbWMF0ka24K3Sn,https://api.spotify.com/v1/tracks/5YltN2bsWbWM...,https://api.spotify.com/v1/audio-analysis/5Ylt...,248987.0,4.0
3,0.491,0.743,2.0,-5.550,1.0,0.0319,0.15400,0.000,0.3080,0.3600,141.942,audio_features,3WibkuoQf13pzeXfHO4hmT,spotify:track:3WibkuoQf13pzeXfHO4hmT,https://api.spotify.com/v1/tracks/3WibkuoQf13p...,https://api.spotify.com/v1/audio-analysis/3Wib...,234547.0,4.0
4,0.392,0.613,7.0,-7.465,1.0,0.0389,0.05500,0.000,0.9610,0.1880,144.598,audio_features,5cxT7mgc8TSxOHiLfwqCwI,spotify:track:5cxT7mgc8TSxOHiLfwqCwI,https://api.spotify.com/v1/tracks/5cxT7mgc8TSx...,https://api.spotify.com/v1/audio-analysis/5cxT...,257627.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451254,0.538,0.681,1.0,-9.934,1.0,0.0350,0.03910,0.919,0.3620,0.0389,131.990,audio_features,5xLg7ZGyd2OP7d77Qo1Fef,spotify:track:5xLg7ZGyd2OP7d77Qo1Fef,https://api.spotify.com/v1/tracks/5xLg7ZGyd2OP...,https://api.spotify.com/v1/audio-analysis/5xLg...,441794.0,4.0
1451255,0.467,0.782,1.0,-9.929,1.0,0.0506,0.02420,0.817,0.1100,0.1740,192.248,audio_features,3ZQi08wWARdgXeErK0qaaS,spotify:track:3ZQi08wWARdgXeErK0qaaS,https://api.spotify.com/v1/tracks/3ZQi08wWARdg...,https://api.spotify.com/v1/audio-analysis/3ZQi...,260910.0,4.0
1451256,0.549,0.799,5.0,-6.213,0.0,0.0739,0.03970,0.872,0.1270,0.4900,120.000,audio_features,2csf4EmM3Ryd1dOPBZUqJm,spotify:track:2csf4EmM3Ryd1dOPBZUqJm,https://api.spotify.com/v1/tracks/2csf4EmM3Ryd...,https://api.spotify.com/v1/audio-analysis/2csf...,288250.0,4.0
1451257,0.444,0.434,10.0,-10.242,0.0,0.0558,0.00282,0.802,0.1430,0.0413,95.031,audio_features,4VbXzK9CxGX5cjtxu8cUmk,spotify:track:4VbXzK9CxGX5cjtxu8cUmk,https://api.spotify.com/v1/tracks/4VbXzK9CxGX5...,https://api.spotify.com/v1/audio-analysis/4VbX...,236409.0,4.0


In [7]:
audio_features_df.rename(columns={'uri': 'track_uri'}, inplace=True)
audio_features_df

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,track_uri,track_href,analysis_url,duration_ms,time_signature
0,0.51,0.620,4.0,-6.269,1.0,0.0298,0.17200,0.000,0.0983,0.3160,136.203,audio_features,3KRIjXjUYoiweWF8x8Eg6v,spotify:track:3KRIjXjUYoiweWF8x8Eg6v,https://api.spotify.com/v1/tracks/3KRIjXjUYoiw...,https://api.spotify.com/v1/audio-analysis/3KRI...,256960.0,4.0
1,0.468,0.739,2.0,-6.123,1.0,0.0328,0.06120,0.000,0.2020,0.3710,155.984,audio_features,4TMy1kRMymO1iBNdqnMDGE,spotify:track:4TMy1kRMymO1iBNdqnMDGE,https://api.spotify.com/v1/tracks/4TMy1kRMymO1...,https://api.spotify.com/v1/audio-analysis/4TMy...,236387.0,3.0
2,0.38,0.440,4.0,-7.743,0.0,0.0612,0.44400,0.000,0.1510,0.1750,71.335,audio_features,5YltN2bsWbWMF0ka24K3Sn,spotify:track:5YltN2bsWbWMF0ka24K3Sn,https://api.spotify.com/v1/tracks/5YltN2bsWbWM...,https://api.spotify.com/v1/audio-analysis/5Ylt...,248987.0,4.0
3,0.491,0.743,2.0,-5.550,1.0,0.0319,0.15400,0.000,0.3080,0.3600,141.942,audio_features,3WibkuoQf13pzeXfHO4hmT,spotify:track:3WibkuoQf13pzeXfHO4hmT,https://api.spotify.com/v1/tracks/3WibkuoQf13p...,https://api.spotify.com/v1/audio-analysis/3Wib...,234547.0,4.0
4,0.392,0.613,7.0,-7.465,1.0,0.0389,0.05500,0.000,0.9610,0.1880,144.598,audio_features,5cxT7mgc8TSxOHiLfwqCwI,spotify:track:5cxT7mgc8TSxOHiLfwqCwI,https://api.spotify.com/v1/tracks/5cxT7mgc8TSx...,https://api.spotify.com/v1/audio-analysis/5cxT...,257627.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1451254,0.538,0.681,1.0,-9.934,1.0,0.0350,0.03910,0.919,0.3620,0.0389,131.990,audio_features,5xLg7ZGyd2OP7d77Qo1Fef,spotify:track:5xLg7ZGyd2OP7d77Qo1Fef,https://api.spotify.com/v1/tracks/5xLg7ZGyd2OP...,https://api.spotify.com/v1/audio-analysis/5xLg...,441794.0,4.0
1451255,0.467,0.782,1.0,-9.929,1.0,0.0506,0.02420,0.817,0.1100,0.1740,192.248,audio_features,3ZQi08wWARdgXeErK0qaaS,spotify:track:3ZQi08wWARdgXeErK0qaaS,https://api.spotify.com/v1/tracks/3ZQi08wWARdg...,https://api.spotify.com/v1/audio-analysis/3ZQi...,260910.0,4.0
1451256,0.549,0.799,5.0,-6.213,0.0,0.0739,0.03970,0.872,0.1270,0.4900,120.000,audio_features,2csf4EmM3Ryd1dOPBZUqJm,spotify:track:2csf4EmM3Ryd1dOPBZUqJm,https://api.spotify.com/v1/tracks/2csf4EmM3Ryd...,https://api.spotify.com/v1/audio-analysis/2csf...,288250.0,4.0
1451257,0.444,0.434,10.0,-10.242,0.0,0.0558,0.00282,0.802,0.1430,0.0413,95.031,audio_features,4VbXzK9CxGX5cjtxu8cUmk,spotify:track:4VbXzK9CxGX5cjtxu8cUmk,https://api.spotify.com/v1/tracks/4VbXzK9CxGX5...,https://api.spotify.com/v1/audio-analysis/4VbX...,236409.0,4.0


In [8]:
track_features_df = merge_files_from_folder(track_features_folder)
track_features_df

,track_uri,release_date,popularity
0,spotify:track:3KRIjXjUYoiweWF8x8Eg6v,2017-02-03,37
1,spotify:track:4TMy1kRMymO1iBNdqnMDGE,2017-02-03,22
2,spotify:track:5YltN2bsWbWMF0ka24K3Sn,2017-02-03,30
3,spotify:track:3WibkuoQf13pzeXfHO4hmT,2017-08-11,28
4,spotify:track:5cxT7mgc8TSxOHiLfwqCwI,2009-11-10,11
...,...,...,...
1478973,spotify:track:5x79FCROLCF6WaxpkeGZu5,2017-06-30,21
1478974,spotify:track:3ouqi6fLYxytZ5eb72h2MU,1985,6
1478975,spotify:track:0A1QItLPaJ91L1w121oWG5,2016-11-08,10
1478976,spotify:track:2B19O2u3vMVVxVuDWI4OLY,2000-04-04,0


In [9]:
artist_features_folder = '/Users/olagavriluk/Desktop/Spotify-Recommendation-System-master/artists'
artist_features_df = merge_files_from_folder(artist_features_folder)
artist_features_df

,artist_uri,artist_pop,genres
0,spotify:artist:5QP6YlOw9QlFgb6pRG6euk,1,unknown
1,spotify:artist:3h0MN1neFknEvlYKxFmSQW,31,bubblegrunge indie_punk indie_surf power-pop_p...
2,spotify:artist:5kxbczSLck59eyR8GDsHd6,18,modern_power_pop
3,spotify:artist:4Xo1N4V7w6qX23OHAPcLCe,17,washington_indie
4,spotify:artist:78inRkIIF0n9R9I2HPoWP7,43,modern_power_pop
...,...,...,...
588776,spotify:artist:1sH2AJxGh9qo0yW3nyjrJO,3,country_rap redneck
588777,spotify:artist:7MhrpXCllfRMB47PtC6Z2F,12,uk_funky
588778,spotify:artist:7e481IDyzSINMX6tH8YCab,29,classical_era
588779,spotify:artist:3FsQX7FsrZAzTU9ah45P7O,0,atl_hip_hop


In [10]:
artist = artist_features_df['artist_uri'].unique()
len(artist)

237418

In [11]:
unique_artist_features_df = artist_features_df.drop_duplicates(subset=['artist_uri'])
unique_artist_features_df

,artist_uri,artist_pop,genres
0,spotify:artist:5QP6YlOw9QlFgb6pRG6euk,1,unknown
1,spotify:artist:3h0MN1neFknEvlYKxFmSQW,31,bubblegrunge indie_punk indie_surf power-pop_p...
2,spotify:artist:5kxbczSLck59eyR8GDsHd6,18,modern_power_pop
3,spotify:artist:4Xo1N4V7w6qX23OHAPcLCe,17,washington_indie
4,spotify:artist:78inRkIIF0n9R9I2HPoWP7,43,modern_power_pop
...,...,...,...
588762,spotify:artist:3of3xznEhG0ksXQkj9NV60,12,unknown
588764,spotify:artist:1cjCX4HD58olazQnmmHSTV,0,unknown
588766,spotify:artist:1ZcMu6OqFhpbGBAkqwQ49x,32,lo-fi_rap
588768,spotify:artist:2OyL7fmRaq36oFkKyjCY6q,26,unknown


In [12]:
art = artist_features_df['artist_uri'].unique()
len(art)

237418

In [13]:
main_df = pd.read_csv('unique_songs_dataset.csv')
main_df

,album,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook
1,0,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone
2,0,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit)
3,0,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified
4,0,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot
...,...,...,...,...,...,...,...,...,...
1483754,948,10,SNAP!,spotify:track:1qjEdWY2RHoVbiCYsbwi26,spotify:artist:2FrKQPjJe4pVMZOgm0ESOx,Exterminate - Michael Gray Remix,spotify:album:7GthPhu4rupbUY0vOQKxJW,428571,SNAP! The Power Greatest Hits
1483755,959,2,Suicidal Tendencies,spotify:track:2kKfbRXlBmaX4A8Lii0SE6,spotify:artist:3WPKDlucMsXH6FC1XaclZC,Born To Be Cyco,spotify:album:5Q9XRmD1Z9IVF4Tlz0MsSK,144013,No Mercy Fool! / The Suicidal Family
1483756,971,35,Disturbing Tha Peace,spotify:track:2rFbG6ltIDAIdawGMjW1oj,spotify:artist:3FsQX7FsrZAzTU9ah45P7O,Break Sumthin',spotify:album:1HjiZwq8HvkzLchkB2XcyV,277560,Golden Grain
1483757,981,19,Arab Strap,spotify:track:2wBZrBqWQ4eIhShITfUJ4c,spotify:artist:6g8Jqb5JMfv92eB2r0awTN,My Favourite Muse,spotify:album:7maGHx2VJGTBPczcWT9JVm,303106,Philophobia


In [14]:
main_df = main_df.merge(audio_features_df, on='track_uri', how='left')
main_df

,album,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms_x,album_name,danceability,...,instrumentalness,liveness,valence,tempo,type,id,track_href,analysis_url,duration_ms_y,time_signature
0,0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,0.904,...,0.006970,0.0471,0.810,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864.0,4.0
1,0,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,0.774,...,0.025000,0.2420,0.924,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800.0,4.0
2,0,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),0.664,...,0.000000,0.0598,0.701,99.252,audio_features,0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933.0,4.0
3,0,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,0.892,...,0.000234,0.0521,0.817,100.972,audio_features,1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267.0,4.0
4,0,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,0.853,...,0.000000,0.3130,0.654,94.759,audio_features,1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483754,948,10,SNAP!,spotify:track:1qjEdWY2RHoVbiCYsbwi26,spotify:artist:2FrKQPjJe4pVMZOgm0ESOx,Exterminate - Michael Gray Remix,spotify:album:7GthPhu4rupbUY0vOQKxJW,428571,SNAP! The Power Greatest Hits,0.74,...,0.726000,0.2060,0.704,125.983,audio_features,1qjEdWY2RHoVbiCYsbwi26,https://api.spotify.com/v1/tracks/1qjEdWY2RHoV...,https://api.spotify.com/v1/audio-analysis/1qjE...,428571.0,4.0
1483755,959,2,Suicidal Tendencies,spotify:track:2kKfbRXlBmaX4A8Lii0SE6,spotify:artist:3WPKDlucMsXH6FC1XaclZC,Born To Be Cyco,spotify:album:5Q9XRmD1Z9IVF4Tlz0MsSK,144013,No Mercy Fool! / The Suicidal Family,0.444,...,0.000000,0.3370,0.223,101.256,audio_features,2kKfbRXlBmaX4A8Lii0SE6,https://api.spotify.com/v1/tracks/2kKfbRXlBmaX...,https://api.spotify.com/v1/audio-analysis/2kKf...,144013.0,4.0
1483756,971,35,Disturbing Tha Peace,spotify:track:2rFbG6ltIDAIdawGMjW1oj,spotify:artist:3FsQX7FsrZAzTU9ah45P7O,Break Sumthin',spotify:album:1HjiZwq8HvkzLchkB2XcyV,277560,Golden Grain,0.856,...,0.000000,0.0727,0.676,145.962,audio_features,2rFbG6ltIDAIdawGMjW1oj,https://api.spotify.com/v1/tracks/2rFbG6ltIDAI...,https://api.spotify.com/v1/audio-analysis/2rFb...,277560.0,4.0
1483757,981,19,Arab Strap,spotify:track:2wBZrBqWQ4eIhShITfUJ4c,spotify:artist:6g8Jqb5JMfv92eB2r0awTN,My Favourite Muse,spotify:album:7maGHx2VJGTBPczcWT9JVm,303106,Philophobia,0.54,...,0.880000,0.1110,0.186,115.493,audio_features,2wBZrBqWQ4eIhShITfUJ4c,https://api.spotify.com/v1/tracks/2wBZrBqWQ4eI...,https://api.spotify.com/v1/audio-analysis/2wBZ...,303107.0,4.0


In [15]:
main_df = main_df.merge(track_features_df, on='track_uri', how='left')
main_df

,album,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms_x,album_name,danceability,...,valence,tempo,type,id,track_href,analysis_url,duration_ms_y,time_signature,release_date,popularity
0,0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,0.904,...,0.810,125.461,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864.0,4.0,2005-07-04,65.0
1,0,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,0.774,...,0.924,143.040,audio_features,6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800.0,4.0,2003-11-13,83.0
2,0,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),0.664,...,0.701,99.252,audio_features,0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933.0,4.0,2003-06-23,15.0
3,0,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,0.892,...,0.817,100.972,audio_features,1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267.0,4.0,2002-11-04,78.0
4,0,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,0.853,...,0.654,94.759,audio_features,1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600.0,4.0,2000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483754,948,10,SNAP!,spotify:track:1qjEdWY2RHoVbiCYsbwi26,spotify:artist:2FrKQPjJe4pVMZOgm0ESOx,Exterminate - Michael Gray Remix,spotify:album:7GthPhu4rupbUY0vOQKxJW,428571,SNAP! The Power Greatest Hits,0.74,...,0.704,125.983,audio_features,1qjEdWY2RHoVbiCYsbwi26,https://api.spotify.com/v1/tracks/1qjEdWY2RHoV...,https://api.spotify.com/v1/audio-analysis/1qjE...,428571.0,4.0,2009-10-19,0.0
1483755,959,2,Suicidal Tendencies,spotify:track:2kKfbRXlBmaX4A8Lii0SE6,spotify:artist:3WPKDlucMsXH6FC1XaclZC,Born To Be Cyco,spotify:album:5Q9XRmD1Z9IVF4Tlz0MsSK,144013,No Mercy Fool! / The Suicidal Family,0.444,...,0.223,101.256,audio_features,2kKfbRXlBmaX4A8Lii0SE6,https://api.spotify.com/v1/tracks/2kKfbRXlBmaX...,https://api.spotify.com/v1/audio-analysis/2kKf...,144013.0,4.0,2010-09-05,0.0
1483756,971,35,Disturbing Tha Peace,spotify:track:2rFbG6ltIDAIdawGMjW1oj,spotify:artist:3FsQX7FsrZAzTU9ah45P7O,Break Sumthin',spotify:album:1HjiZwq8HvkzLchkB2XcyV,277560,Golden Grain,0.856,...,0.676,145.962,audio_features,2rFbG6ltIDAIdawGMjW1oj,https://api.spotify.com/v1/tracks/2rFbG6ltIDAI...,https://api.spotify.com/v1/audio-analysis/2rFb...,277560.0,4.0,2002-01-01,18.0
1483757,981,19,Arab Strap,spotify:track:2wBZrBqWQ4eIhShITfUJ4c,spotify:artist:6g8Jqb5JMfv92eB2r0awTN,My Favourite Muse,spotify:album:7maGHx2VJGTBPczcWT9JVm,303106,Philophobia,0.54,...,0.186,115.493,audio_features,2wBZrBqWQ4eIhShITfUJ4c,https://api.spotify.com/v1/tracks/2wBZrBqWQ4eI...,https://api.spotify.com/v1/audio-analysis/2wBZ...,303107.0,4.0,1998,0.0


In [16]:
main_df = main_df.merge(unique_artist_features_df, on='artist_uri', how='left')
main_df

,album,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms_x,album_name,danceability,...,type,id,track_href,analysis_url,duration_ms_y,time_signature,release_date,popularity,artist_pop,genres
0,0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,0.904,...,audio_features,0UaMYEvWZi0ZqiDOoHU3YI,https://api.spotify.com/v1/tracks/0UaMYEvWZi0Z...,https://api.spotify.com/v1/audio-analysis/0UaM...,226864.0,4.0,2005-07-04,65.0,72.0,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...
1,0,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,0.774,...,audio_features,6I9VzXrHxO9rA9A5euc8Ak,https://api.spotify.com/v1/tracks/6I9VzXrHxO9r...,https://api.spotify.com/v1/audio-analysis/6I9V...,198800.0,4.0,2003-11-13,83.0,83.0,dance_pop pop
2,0,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),0.664,...,audio_features,0WqIKmW4BTrj3eJFmnCKMv,https://api.spotify.com/v1/tracks/0WqIKmW4BTrj...,https://api.spotify.com/v1/audio-analysis/0WqI...,235933.0,4.0,2003-06-23,15.0,87.0,pop r&b
3,0,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,0.892,...,audio_features,1AWQoqb9bSvzTjaLralEkT,https://api.spotify.com/v1/tracks/1AWQoqb9bSvz...,https://api.spotify.com/v1/audio-analysis/1AWQ...,267267.0,4.0,2002-11-04,78.0,83.0,dance_pop pop
4,0,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,0.853,...,audio_features,1lzr43nnXAijIGYnCT8M8H,https://api.spotify.com/v1/tracks/1lzr43nnXAij...,https://api.spotify.com/v1/audio-analysis/1lzr...,227600.0,4.0,2000,0.0,74.0,dance_pop pop_rap reggae_fusion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483754,948,10,SNAP!,spotify:track:1qjEdWY2RHoVbiCYsbwi26,spotify:artist:2FrKQPjJe4pVMZOgm0ESOx,Exterminate - Michael Gray Remix,spotify:album:7GthPhu4rupbUY0vOQKxJW,428571,SNAP! The Power Greatest Hits,0.74,...,audio_features,1qjEdWY2RHoVbiCYsbwi26,https://api.spotify.com/v1/tracks/1qjEdWY2RHoV...,https://api.spotify.com/v1/audio-analysis/1qjE...,428571.0,4.0,2009-10-19,0.0,64.0,diva_house eurodance german_techno hip_house
1483755,959,2,Suicidal Tendencies,spotify:track:2kKfbRXlBmaX4A8Lii0SE6,spotify:artist:3WPKDlucMsXH6FC1XaclZC,Born To Be Cyco,spotify:album:5Q9XRmD1Z9IVF4Tlz0MsSK,144013,No Mercy Fool! / The Suicidal Family,0.444,...,audio_features,2kKfbRXlBmaX4A8Lii0SE6,https://api.spotify.com/v1/tracks/2kKfbRXlBmaX...,https://api.spotify.com/v1/audio-analysis/2kKf...,144013.0,4.0,2010-09-05,0.0,56.0,alternative_metal california_hardcore crossove...
1483756,971,35,Disturbing Tha Peace,spotify:track:2rFbG6ltIDAIdawGMjW1oj,spotify:artist:3FsQX7FsrZAzTU9ah45P7O,Break Sumthin',spotify:album:1HjiZwq8HvkzLchkB2XcyV,277560,Golden Grain,0.856,...,audio_features,2rFbG6ltIDAIdawGMjW1oj,https://api.spotify.com/v1/tracks/2rFbG6ltIDAI...,https://api.spotify.com/v1/audio-analysis/2rFb...,277560.0,4.0,2002-01-01,18.0,0.0,atl_hip_hop
1483757,981,19,Arab Strap,spotify:track:2wBZrBqWQ4eIhShITfUJ4c,spotify:artist:6g8Jqb5JMfv92eB2r0awTN,My Favourite Muse,spotify:album:7maGHx2VJGTBPczcWT9JVm,303106,Philophobia,0.54,...,audio_features,2wBZrBqWQ4eIhShITfUJ4c,https://api.spotify.com/v1/tracks/2wBZrBqWQ4eI...,https://api.spotify.com/v1/audio-analysis/2wBZ...,303107.0,4.0,1998,0.0,36.0,scottish_indie slowcore


In [20]:
main_df.columns

Index(['album', 'pos', 'artist_name', 'track_uri', 'artist_uri', 'track_name',
       'album_uri', 'duration_ms_x', 'album_name', 'danceability', 'energy',
       'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id',
       'track_href', 'analysis_url', 'duration_ms_y', 'time_signature',
       'release_date', 'popularity', 'artist_pop', 'genres'],
      dtype='object')

In [17]:
df = pd.read_csv('1m.csv')

In [18]:
df

,Unnamed: 0,album,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name
0,0,0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook
1,1,0,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone
2,2,0,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit)
3,3,0,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified
4,4,0,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot
...,...,...,...,...,...,...,...,...,...,...
2262287,2262287,981,19,Arab Strap,spotify:track:2wBZrBqWQ4eIhShITfUJ4c,spotify:artist:6g8Jqb5JMfv92eB2r0awTN,My Favourite Muse,spotify:album:7maGHx2VJGTBPczcWT9JVm,303106,Philophobia
2262288,2262288,985,2,Various Artists,spotify:track:2t7fotSpsiWHpSTySbSNZg,spotify:artist:0LyfQWJT6nXafLPZqxe9Of,The Stars and Stripes Forever,spotify:album:5HuTwuPKjQKNpmLmmi4Njx,314644,My Kids Salute Our Troops
2262289,2262289,998,3,Faragó,spotify:track:0EW4RltERtn276lOi0DXQj,spotify:artist:6x3HJm6n40OUW2ZcmttBxQ,Killer,spotify:album:5aauU2AK4iG01BxIchws8Q,297176,Downtown / Archipelago
2262290,2262290,998,71,Crazy Fool,spotify:track:5RiCId3jJs8D9g6Fv4A1Su,spotify:artist:4UrrAgW6WW7UfZ760eyHe7,Robin Hood,spotify:album:2MUG8V88S5mazH8n45AKuZ,278973,Train of Thought


In [19]:
ar = df['artist_uri'].unique()
len(ar)

295860

In [21]:
unique_songs_df = main_df

unique_songs_df = unique_songs_df[['track_name', 'danceability', 'energy', 'key', 
                                   'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 
                                   'liveness', 'valence', 'tempo', 'time_signature', 
                                   'release_date', 'popularity', 'artist_pop', 'genres']]
merged_df = df.merge(unique_songs_df, on='track_name', how='left')

In [22]:
merged_df

,Unnamed: 0,album,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_date,popularity,artist_pop,genres
0,0,0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,...,0.03110,0.006970,0.0471,0.8100,125.461,4.0,2005-07-04,65.0,72.0,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...
1,1,0,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,...,0.02490,0.025000,0.2420,0.9240,143.040,4.0,2003-11-13,83.0,83.0,dance_pop pop
2,2,0,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),...,0.00238,0.000000,0.0598,0.7010,99.252,4.0,2003-06-23,15.0,87.0,pop r&b
3,3,0,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,...,0.20100,0.000234,0.0521,0.8170,100.972,4.0,2002-11-04,78.0,83.0,dance_pop pop
4,4,0,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,...,0.05610,0.000000,0.3130,0.6540,94.759,4.0,2000,0.0,74.0,dance_pop pop_rap reggae_fusion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2262287,2262287,981,19,Arab Strap,spotify:track:2wBZrBqWQ4eIhShITfUJ4c,spotify:artist:6g8Jqb5JMfv92eB2r0awTN,My Favourite Muse,spotify:album:7maGHx2VJGTBPczcWT9JVm,303106,Philophobia,...,0.61400,0.880000,0.1110,0.1860,115.493,4.0,1998,0.0,36.0,scottish_indie slowcore
2262288,2262288,985,2,Various Artists,spotify:track:2t7fotSpsiWHpSTySbSNZg,spotify:artist:0LyfQWJT6nXafLPZqxe9Of,The Stars and Stripes Forever,spotify:album:5HuTwuPKjQKNpmLmmi4Njx,314644,My Kids Salute Our Troops,...,0.96200,0.671000,0.1730,0.8490,121.145,4.0,2004-01-01,16.0,14.0,military_band
2262289,2262289,998,3,Faragó,spotify:track:0EW4RltERtn276lOi0DXQj,spotify:artist:6x3HJm6n40OUW2ZcmttBxQ,Killer,spotify:album:5aauU2AK4iG01BxIchws8Q,297176,Downtown / Archipelago,...,0.60800,0.000000,0.3050,0.5620,69.964,4.0,2010-01-01,14.0,64.0,modern_rock neo_mellow neon_pop_punk pop_punk ...
2262290,2262290,998,71,Crazy Fool,spotify:track:5RiCId3jJs8D9g6Fv4A1Su,spotify:artist:4UrrAgW6WW7UfZ760eyHe7,Robin Hood,spotify:album:2MUG8V88S5mazH8n45AKuZ,278973,Train of Thought,...,0.00493,0.000059,0.1680,0.0848,113.939,4.0,2016-11-07,0.0,46.0,canadian_hip_hop underground_hip_hop


In [23]:
merged_df.columns

Index(['Unnamed: 0', 'album', 'pos', 'artist_name', 'track_uri', 'artist_uri',
       'track_name', 'album_uri', 'duration_ms', 'album_name', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature',
       'release_date', 'popularity', 'artist_pop', 'genres'],
      dtype='object')

In [24]:
merged_df.isna().sum()

Unnamed: 0              0
album                   0
pos                     0
artist_name            10
track_uri               0
artist_uri              0
track_name              7
album_uri               0
duration_ms             0
album_name              7
danceability        43665
energy              43665
key                 43665
loudness            43665
mode                43665
speechiness         43665
acousticness        43665
instrumentalness    43665
liveness            43665
valence             43665
tempo               43665
time_signature      43665
release_date         6345
popularity           6345
artist_pop           3475
genres               3475
dtype: int64

In [26]:
try_df = merged_df

In [27]:
try_df.dropna(axis=0,inplace=True)

In [29]:
try_df.isna().sum().sum()

0

In [33]:
t = try_df['album_uri'].unique()
len(t)

723909

In [34]:
try_df

,Unnamed: 0,album,pos,artist_name,track_uri,artist_uri,track_name,album_uri,duration_ms,album_name,...,acousticness,instrumentalness,liveness,valence,tempo,time_signature,release_date,popularity,artist_pop,genres
0,0,0,0,Missy Elliott,spotify:track:0UaMYEvWZi0ZqiDOoHU3YI,spotify:artist:2wIVse2owClT7go1WT98tk,Lose Control (feat. Ciara & Fat Man Scoop),spotify:album:6vV5UrXcfyQD1wu4Qo2I9K,226863,The Cookbook,...,0.03110,0.006970,0.0471,0.8100,125.461,4.0,2005-07-04,65.0,72.0,dance_pop hip_hop hip_pop neo_soul pop_rap r&b...
1,1,0,1,Britney Spears,spotify:track:6I9VzXrHxO9rA9A5euc8Ak,spotify:artist:26dSoYclwsYLMAKD3tpOr4,Toxic,spotify:album:0z7pVBGOD7HCIB7S8eLkLI,198800,In The Zone,...,0.02490,0.025000,0.2420,0.9240,143.040,4.0,2003-11-13,83.0,83.0,dance_pop pop
2,2,0,2,Beyoncé,spotify:track:0WqIKmW4BTrj3eJFmnCKMv,spotify:artist:6vWDO969PvNqNYHIOW5v0m,Crazy In Love,spotify:album:25hVFAxTlDvXbx2X2QkUkE,235933,Dangerously In Love (Alben für die Ewigkeit),...,0.00238,0.000000,0.0598,0.7010,99.252,4.0,2003-06-23,15.0,87.0,pop r&b
3,3,0,3,Justin Timberlake,spotify:track:1AWQoqb9bSvzTjaLralEkT,spotify:artist:31TPClRtHm23RisEBtV3X7,Rock Your Body,spotify:album:6QPkyl04rXwTGlGlcYaRoW,267266,Justified,...,0.20100,0.000234,0.0521,0.8170,100.972,4.0,2002-11-04,78.0,83.0,dance_pop pop
4,4,0,4,Shaggy,spotify:track:1lzr43nnXAijIGYnCT8M8H,spotify:artist:5EvFsr3kj42KNv97ZEnqij,It Wasn't Me,spotify:album:6NmFmPX56pcLBOFMhIiKvF,227600,Hot Shot,...,0.05610,0.000000,0.3130,0.6540,94.759,4.0,2000,0.0,74.0,dance_pop pop_rap reggae_fusion
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2262287,2262287,981,19,Arab Strap,spotify:track:2wBZrBqWQ4eIhShITfUJ4c,spotify:artist:6g8Jqb5JMfv92eB2r0awTN,My Favourite Muse,spotify:album:7maGHx2VJGTBPczcWT9JVm,303106,Philophobia,...,0.61400,0.880000,0.1110,0.1860,115.493,4.0,1998,0.0,36.0,scottish_indie slowcore
2262288,2262288,985,2,Various Artists,spotify:track:2t7fotSpsiWHpSTySbSNZg,spotify:artist:0LyfQWJT6nXafLPZqxe9Of,The Stars and Stripes Forever,spotify:album:5HuTwuPKjQKNpmLmmi4Njx,314644,My Kids Salute Our Troops,...,0.96200,0.671000,0.1730,0.8490,121.145,4.0,2004-01-01,16.0,14.0,military_band
2262289,2262289,998,3,Faragó,spotify:track:0EW4RltERtn276lOi0DXQj,spotify:artist:6x3HJm6n40OUW2ZcmttBxQ,Killer,spotify:album:5aauU2AK4iG01BxIchws8Q,297176,Downtown / Archipelago,...,0.60800,0.000000,0.3050,0.5620,69.964,4.0,2010-01-01,14.0,64.0,modern_rock neo_mellow neon_pop_punk pop_punk ...
2262290,2262290,998,71,Crazy Fool,spotify:track:5RiCId3jJs8D9g6Fv4A1Su,spotify:artist:4UrrAgW6WW7UfZ760eyHe7,Robin Hood,spotify:album:2MUG8V88S5mazH8n45AKuZ,278973,Train of Thought,...,0.00493,0.000059,0.1680,0.0848,113.939,4.0,2016-11-07,0.0,46.0,canadian_hip_hop underground_hip_hop


In [ ]:
df.to_csv('final.csv', index=False)